[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://drive.google.com/file/d/1z5sZPtM8r73YPEwQKmYi6Q6rYAufCKWp/view?usp=sharing)

Download [TKO](http://tko.ccbr.utoronto.ca/) HeLa data.

In [3]:
! wget http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib1.gz
! wget http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib2.gz
! gunzip readcount-HeLa-lib1.gz -y
! gunzip readcount-HeLa-lib2.gz -y

--2023-04-10 22:36:02--  http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib1.gz
Resolving tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)... 142.150.76.126
Connecting to tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)|142.150.76.126|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2877516 (2.7M) [application/x-gzip]
Saving to: ‘readcount-HeLa-lib1.gz.1’

100%[======================================>] 2,877,516   8.58MB/s   in 0.3s   

2023-04-10 22:36:03 (8.58 MB/s) - ‘readcount-HeLa-lib1.gz.1’ saved [2877516/2877516]

--2023-04-10 22:36:03--  http://tko.ccbr.utoronto.ca/Data/readcount-HeLa-lib2.gz
Resolving tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)... 142.150.76.126
Connecting to tko.ccbr.utoronto.ca (tko.ccbr.utoronto.ca)|142.150.76.126|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2680113 (2.6M) [application/x-gzip]
Saving to: ‘readcount-HeLa-lib2.gz.1’

100%[======================================>] 2,680,113   14.8MB/s   in 0.2s   

2023

In [4]:
! head readcount-HeLa-lib1

GENE_CLONE	GENE	T08A	T08B	T08C	T12A	T12B	T12C	T15A	T15B	T15C	T18A	T18B	T18C	T0
A1BG_CACCTTCGAGCTGCTGCGCG	A1BG	310	226	338	356	249	224	186	60	296	125	49	296	469
A1BG_AAGAGCGCCTCGGTCCCAGC	A1BG	46	1	0	7	22	142	0	1	52	0	1	52	213
A1BG_TGGACTTCCAGCTACGGCGC	A1BG	239	216	285	117	244	116	172	298	269	119	250	269	363
A1BG_CACTGGCGCCATCGAGAGCC	A1BG	289	83	166	164	111	14	184	160	214	122	137	214	678
A1BG_GCTCGGGCTTGTCCACAGGA	A1BG	205	34	217	205	148	355	326	100	432	212	85	432	559
A1BG_CAAGAGAAAGACCACGAGCA	A1BG	389	331	468	1074	364	158	664	286	499	464	235	499	647
A1CF_CGTGGCTATTTGGCATACAC	A1CF	452	240	390	630	509	261	471	255	301	322	210	301	898
A1CF_GGTATACTCTCCTTGCAGCA	A1CF	71	30	29	119	155	153	131	76	56	94	61	56	199
A1CF_GACATGGTATTGCAGTAGAC	A1CF	207	227	223	118	141	173	176	198	42	118	166	42	271


In [5]:
! pip install anndata

Looking in indexes: https://pypi.org/simple, https://packagecloud.io/github/git-lfs/pypi/simple


In [6]:
import pandas as pd
import anndata as ad

In [12]:
def make_anndata(tbl):
  sample_df = pd.DataFrame(tbl.columns[2:]).rename(columns={0:"sample"}).set_index("sample")
  sample_df["replicate"] = sample_df.index.str[-1]
  sample_df["time"] = sample_df.index.str[1:-1].map(lambda s: int(s) if s else -1)
  return ad.AnnData(X=tbl.values[:,2:].T, var=tbl.iloc[:,:2], 
                   obs=sample_df)

In [13]:
adata = make_anndata(tbl)
bdata = make_anndata(tbl2)

/tmp/ipykernel_2520/951100887.py:5: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return ad.AnnData(X=tbl.values[:,2:].T, var=tbl.iloc[:,:2],
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/igvf_sccrispr/lib/python3.10/site-packages/anndata/_core/anndata.py:121: ImplicitModificationWarning: Transforming to str index.
  warnings.warn("Transforming to str index.", ImplicitModificationWarning)
/tmp/ipykernel_2520/951100887.py:5: FutureWarning: X.dtype being converted to np.float32 from object. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  return ad.AnnData(X=tbl.values[:,2:].T, var=tbl.iloc[:,:2],
/data/pinello/SHARED_SOFTWARE/anaconda_latest/envs/ig

In [14]:
adata

AnnData object with n_obs × n_vars = 13 × 91320
    obs: 'replicate', 'time'
    var: 'GENE_CLONE', 'GENE'

In [15]:
adata.obs

,replicate,time
sample,,
T08A,A,8
T08B,B,8
T08C,C,8
T12A,A,12
T12B,B,12
T12C,C,12
T15A,A,15
T15B,B,15
T15C,C,15


In [16]:
adata.var

,GENE_CLONE,GENE
0,A1BG_CACCTTCGAGCTGCTGCGCG,A1BG
1,A1BG_AAGAGCGCCTCGGTCCCAGC,A1BG
2,A1BG_TGGACTTCCAGCTACGGCGC,A1BG
3,A1BG_CACTGGCGCCATCGAGAGCC,A1BG
4,A1BG_GCTCGGGCTTGTCCACAGGA,A1BG
...,...,...
91315,luciferase_CCTCTAGAGGATGGAACCGC,luciferase
91316,luciferase_ACAACTTTACCGACCGCGCC,luciferase
91317,luciferase_CTTGTCGTATCCCTGGAAGA,luciferase
91318,luciferase_GGCTATGAAGAGATACGCCC,luciferase


In [17]:
bdata

AnnData object with n_obs × n_vars = 13 × 85180
    obs: 'replicate', 'time'
    var: 'GENE_CLONE', 'GENE'

In [18]:
bdata.obs

,replicate,time
sample,,
T08A,A,8
T08B,B,8
T08C,C,8
T12A,A,12
T12B,B,12
T12C,C,12
T15A,A,15
T15B,B,15
T15C,C,15


### Slicing

In [20]:
adata_cut = adata[:, adata.var.GENE == "A1BG"]
adata_cut

View of AnnData object with n_obs × n_vars = 13 × 6
    obs: 'replicate', 'time'
    var: 'GENE_CLONE', 'GENE'

In [21]:
adata_t8 = adata[adata.obs.time == 8, :]
adata_t8

View of AnnData object with n_obs × n_vars = 3 × 91320
    obs: 'replicate', 'time'
    var: 'GENE_CLONE', 'GENE'

### Writing

In [22]:
adata.write("HeLa_lib1.h5ad")